In [1]:
! pip install sqlalchemy
! pip install PyMySQL

You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

data = pd.read_sql_query('SELECT * FROM rental', engine)
data.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [4]:
#Q7 a
query_result_may = engine.execute('select customer_id, count(rental_id) from rentals_may group by customer_id order by customer_id') # watch out: it's necessary to have this in the cell (result is not saved)
result_may = pd.DataFrame(query_result_may) 
result_may = result_may.set_index(0)
result_may


,1
0,
1,2
2,1
3,2
5,3
6,3
...,...
594,4
595,1
596,6


In [5]:
#Q7 b
query_result_june = engine.execute('select customer_id, count(rental_id) from rentals_june group by customer_id order by customer_id') # watch out: it's necessary to have this in the cell (result is not saved)
result_june = pd.DataFrame(query_result_june) 
result_june = result_june.set_index(0)
result_june
#query_result_june

,1
0,
1,7
2,1
3,4
4,6
5,5
...,...
595,2
596,2
597,3


In [6]:
df = pd.merge(result_may, result_june,
              left_index=True, right_index=True, how = 'outer')
df


,1_x,1_y
0,,
1,2.0,7.0
2,1.0,1.0
3,2.0,4.0
4,NaN,6.0
5,3.0,5.0
...,...,...
595,1.0,2.0
596,6.0,2.0
597,2.0,3.0


In [7]:
df = df.rename(columns={'1_x':'May_Cnt',
                        '1_y':'June_Cnt'})

df.columns

Index(['May_Cnt', 'June_Cnt'], dtype='object')

In [8]:
df['May_Cnt'] = df['May_Cnt'].fillna(0)
df['June_Cnt'] = df['June_Cnt'].fillna(0)
df

,May_Cnt,June_Cnt
0,,
1,2.0,7.0
2,1.0,1.0
3,2.0,4.0
4,0.0,6.0
5,3.0,5.0
...,...,...
595,1.0,2.0
596,6.0,2.0
597,2.0,3.0


In [9]:
#Q 8
def more_films(customer_id,df):
    df1 = df.filter(items=[customer_id], axis=0)
    if float(df1.May_Cnt) > float(df1.June_Cnt):
        return "Customer bought more in May"
    else:
        return"Customer bought more in June" 
    

In [10]:
more_films(596, df)

'Customer bought more in May'